### Parser (Procedures only)

Concrete grammar: 

    expression ::= relation ["≡" relation]
    relation ::= term [ "→" term]
    term ::= factor [ ("∧" | "∨" ) factor]
    factor ::= identifier 
        | "true" 
        | "false" 
        | "¬" expression
        | "∀" expression
        | "∃" expression
        | "(" expression ")" 

Procedure `expression()` parses

    expression(e) ::=
        relation(e)
        [ "≡ relation(r) «e := Equiv(EQUIVALENT, e, r)» ]
   

and returns an abstract syntax tree node or raises an exception with the error.

In [ ]:
def expression():
    if sym in (TRUE, FALSE, AND, OR, LPAREN, RPAREN, NOT, IDENT, EQUIVALENT, IMPLIES, FORALL, EXIST, BAR, DOT):
        e = relation()
        while sym == EQUIVALENT:
            op = sym; getSym()
            e = Equiv(op, e, relation())
        return e
    else: error("expression expected")
        

Procedure `relation()` parses

    relation(r) ::=
        term(r)
        ["→" term(t) «r := Imply(IMPLIES, r, t)» ]

and returns an abstract syntax tree node.

In [ ]:
def relation():
    r = term()
    while sym == IMPLIES:
        op = sym; getSym()
        r = Imply(op, r, term())
    return r


Procedure `term()` parses

    term(t) ::=
        factor(t)
        ["∧" factor(f) «r := And(AND, t, f)» |
          "∨" factor(f) «r := Or(OR, t, f)» ]

and returns an abstract syntax tree node.

In [ ]:
def term():
    t = factor()
    while sym in (AND, OR):
        op = sym; getSym()
        if op == AND:
            t = And(op, t, factor())
        elif op == OR:
            t = Or(op, t, factor())
    return t


Procedure `factor()` parses

    factor(f) ::=
        identifier(name) «f := Ident(name)» |
        "true" «f := true» |
        "false" «f := false» |
        "not" expression(f) «f := not f» |
        "forall" expression(f) «f := forall f» |
        "exist" expression(f) «f := exist f» |
        "(" expression(f) ")"
        

and returns an abstract syntax tree node or raises an exception with the error.

In [4]:
def factor():
    if sym == TRUE: f = True; getSym()
    elif sym == FALSE: f = False; getSym()
    elif sym == LPAREN:
        getSym(); f = expression()
        if sym == RPAREN: getSym()
        else: error(') missing')
    elif sym == NOT:
        getSym(); f = Not(NOT, expression())
    elif sym == IDENT:
        name = val; getSym()
        f = Ident(name)
    elif sym == FORALL:
        getSym()
        q = expression()
        if sym != BAR:
            error('Bar expected')
        getSym(); r = expression()
        if sym != DOT:
            error('Dot expected')
        getSym(); t = expression()
        f = Forall(FORALL, q, r, t)
    elif sym == EXIST:
        getSym()
        q = expression()
        if sym != BAR:
            error('Bar expected')
        getSym(); r = expression()
        if sym != DOT:
            error('Dot expected')
        getSym(); t = expression()
        f = Exist(EXIST, q, r, t)
    else: error('unexpected symbol')
    return f 
